# Proyecto Grupo 5 - Análisis Musical en Spotify

# Librerias y Config

In [1]:
import datetime
import typing

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# spotify.sqlite

`spotify.sqlite` es el archivo con el que partimos este proyecto, obtenido de [los siguientes datos de kaggle](https://www.kaggle.com/datasets/maltegrosse/8-m-spotify-tracks-genre-audio-features). Contiene millones de canciones junto al analisis de caracteristicas que provee la api de spotify.

### Cargando datos

Estos se encuentran contenidos en una BDD `spotify.sqlite`, la cual tendremos en una carpeta `data`.

In [ ]:
import sqlite3

# funcion para cargar las tablas de la base de datos en un diccionario de dataframes
def load_data():
  # conexion a la base de datos
  conn = sqlite3.connect(
    database='data/spotify.sqlite'
  )
  # para que no haya problemas con los caracteres especiales
  conn.text_factory = lambda b: b.decode(encoding = 'utf-8', errors = 'ignore')
  
  # lista de tablas de la base de datos
  tables = [
    "r_albums_artists",
    "r_albums_tracks",
    "r_artist_genre",
    "r_track_artist",
    "genres",
    "albums",
    "artists",
    "audio_features",
    "tracks",
  ]

  # diccionario de dataframes
  dataframes : typing.Dict[str, pd.DataFrame] = {}
  
  # cargar cada tabla en un dataframe
  for table in tables:
    dataframes[table] = pd.read_sql_query(
      sql = f"SELECT * FROM {table}",
      con = conn
    )
  
  # cerrar la conexion a la base de datos
  conn.close()

  return dataframes

# ejecutar la funcion
dataframes : typing.Dict[str, pd.DataFrame] = load_data()
[
  tracks, artists, albums, genres, audio_features, 
  r_albums_artists, r_albums_tracks, r_artist_genre, r_track_artist
] = [
  dataframes['tracks'], dataframes['artists'], dataframes['albums'], dataframes['genres'], dataframes['audio_features'] ,
  dataframes['r_albums_artists'], dataframes['r_albums_tracks'], dataframes['r_artist_genre'], dataframes['r_track_artist']
]
# ver las tablas cargadas
dataframes.keys()

### Formato de las tablas

In [ ]:
info_df = pd.DataFrame()
info_df['tabla'] = dataframes.keys()
info_df['filas'] = [dataframes[i].shape[0] for i in dataframes.keys()]
info_df['columnas'] = [dataframes[i].shape[1] for i in dataframes.keys()]
info_df['nombre_columnas'] = [list(dataframes[i].columns) for i in dataframes.keys()]

info_df.set_index('tabla', inplace=True)
info_df

### Estadisticas y exploración inicial

Las tablas `r_albums_artists`, `r_albums_tracks`, `r_artist_genre` solamente contienen ids, por lo que no contienen estadisticas interesantes

#### albums

In [ ]:
albums.head()

In [ ]:
albums.shape

In [ ]:
albums.info()

In [ ]:
albums.describe()

En base a las celdas anteriores, consideramos: 

- Formalizar `release_date`, estan en formato timestamp unix, por lo que seria conveniente tener esa informacion como fecha. \
  Ademas debemos tener cuidado con los timestamps negativos, los que nos pueden traer problemas (algunos van al año 0)
- Analizar si la popularidad del album es importante, pues 75% de sus valores es 0 en un rango de 0 a 100


In [ ]:
albums['album_type'].value_counts()

In [ ]:
albums['album_group'].value_counts()

In [ ]:
(albums['album_group'] == '').value_counts()

En base a las celdas anteriores, consideramos: 

- `album_group`, que solo contiene strings vacios, es candidato a eliminar.

#### artists

In [ ]:
artists.head()

In [ ]:
artists.shape

In [ ]:
artists.info()

In [ ]:
artists.describe()

Aqui no consideramos modificar los datos. Pero hay que considerar un analisis mas en profundidad de las columnas de popularidad y seguidores.

#### tracks

In [ ]:
tracks.head()

In [ ]:
tracks.shape

In [ ]:
tracks.info()

In [ ]:
tracks.describe()

A partir de las celdas anteriores, consideramos:

- `disc_number` es candidato a eliminar, pues tiene poca varianza y algunos outliers.
- Analizar si `track_number` puede ser una columna util o no. Parece tener outliers (ver `max`).
- `is_playable` es candidato a eliminar, pues su count indica que muchos de sus valores son nulos.

In [ ]:
tracks['disc_number'].value_counts()

In [ ]:
tracks['disc_number'].max()

In [ ]:
tracks['disc_number'].var()

In [ ]:
tracks['track_number'].value_counts()

OPAAA, que paso ahi?
investiguemos 😎

In [ ]:
tracks['track_number'].max()

In [ ]:
tracks[tracks['track_number'] >= 1000].sample(5)

Curioso, estas canciones estan relacionadas y tienen su numero de **_Chapter_** como `track_number`, esto nos genera una serie de outliers que posiblemente van de track number 1 a 1522. Extendamos la busqueda.

In [ ]:
# obtenemos todos los tracks del album que contiene el track con el numero mas alto
r_albums_tracks[
  r_albums_tracks['album_id'] == r_albums_tracks[
    # album que tiene el track con el numero mas alto
    r_albums_tracks['track_id'] == tracks[
      # track con el numero mas alto
      tracks['track_number'] == tracks['track_number'].max()
    ]['id'].values[0]
  ]['album_id'].values[0]
].shape

Exactamente lo que pensabamos, tenemos nuestro primer outlier. Utilizemos `track_number` como filtro para ver si encontramos otros.

In [ ]:
tracks[tracks['track_number'] >= 500].sample(5)

En efecto, Aparece nuevamente el conde de montecristo, pero ademas comenzamos a encontrar audios de la biblia (?), efectos de sonido y canciones de musica clasica (posiblemente de albumes recopilatorios).

En base a esto podemos definir que `track_number` nos puede servir como filtro a todos estos datos que no nos sean utiles y que quizas podemos excluir los albumes recopilatorios de los datos.

Finalmente, hay que decidir donde pondremos nuestro limite a considerar y si excluimos los albumes recopilatorios de los datos.

In [ ]:
tracks['is_playable'].value_counts()

Ya habiamos visto que a esta columna le faltaban muchos datos, comprobemos cuantos son nulos.

In [ ]:
tracks[tracks['is_playable'].isna()].shape

Hay muchos nulos en esta columna!!! Claramente es eliminable.

In [ ]:
tracks['explicit'].value_counts()

In [ ]:
tracks['duration'].value_counts()

La duracion tambien evidencia presencia de algunos outliers, sin embargo, conviene excluir a los albumes recopilatorios del analisis y verificar nuevamente.

#### audio_features

In [ ]:
audio_features.head()

In [ ]:
audio_features.shape

In [ ]:
audio_features.info()

In [ ]:
audio_features.describe()

Aqui solo eliminaremos `analysis_url` (no nos sirve sin un token), pues es la tabla con la informacion de mayor interes.

#### genres

In [ ]:
genres.head()

In [ ]:
genres.shape

In [ ]:
genres.info()

In [ ]:
genres.value_counts()

Esta tabla va relacionada al artista, no a las canciones o albumes, por lo que hay que considerar si es realmente util o no.

### Eliminar y modificar columnas

#### albums

Inicialmente, modificaremos `release_date` para que sea en formato de fecha y no timestamp, luego eliminamos `album_group` debido a que todos son valores vacios.

In [ ]:
# debido a limitaciones en los timestamps de pandas, utilizamos la clase datetime de python (la columna será de tipo object para pandas)
# no tan fun fact: pd.Timestamp.min es el 1 de enero de 1677 :(

# Para rematar, fromtimestamp() no acepta fechas negativas, por lo que tenemos que hacer un workaround con timedelta

# funcion para convertir un timestamp a una fecha
def convert_timestamp_to_date(timestamp):
  try:
    # pasamos el timestamp a dias y luego a fecha
    return (datetime.datetime.fromtimestamp(0) + datetime.timedelta(days = timestamp / 1000 / 60 / 60 / 24)).date()
  # existe un limite para las fechas negativas, lo que nos da un error de OverflowError
  except OverflowError:
    # si el timestamp es positivo, devolvemos la fecha maxima
    if timestamp > 0:
      return datetime.datetime.max.date()
    # si el timestamp es negativo, devolvemos la fecha minima
    else:
      return datetime.datetime.min.date()

In [ ]:
# pasar release_date de timestamp a datetime
albums['album_release_date'] = albums['release_date'].apply(
  lambda x: convert_timestamp_to_date(x)
)
albums['album_release_year'] = albums['album_release_date'].apply(
  lambda x: x.year
)
albums['album_release_month'] = albums['album_release_date'].apply(
  lambda x: x.month
)
# OJO: probablemente no sea muy preciso
albums['album_release_day'] = albums['album_release_date'].apply(
  lambda x: x.day
)

albums.drop(columns=[
  'album_group', 'release_date',
], inplace=True)
albums.rename(columns={
  'id': 'album_id',
  'name': 'album_name',
  'popularity': 'album_popularity',
}, inplace=True)
albums.head()

In [ ]:
albums.describe().apply(lambda s: s.apply(lambda x: format(x, '.0f')))

#### artists

In [ ]:
artists.rename(columns={
  'id': 'artist_id',
  'name': 'artist_name',
  'popularity': 'artist_popularity',
  'followers': 'artist_followers',
}, inplace=True)
artists.head()

#### tracks

Inicialmente, consideramos eliminar `disc_number` por baja varianza y `is_playable` por muchos valores nulos.
Ademas cambiaremos algunos nombres de columnas.

In [ ]:
tracks.drop(columns=[
  'preview_url', 'disc_number', 'is_playable'
], inplace=True)
tracks.rename(columns={
  'id': 'track_id',
  'name': 'track_name',
  'duration': 'track_duration_ms',
  'track_number': 'track_number_in_album',
  'explicit': 'track_explicit',
  'popularity': 'track_popularity'
}, inplace=True)
tracks.head()

#### audio_features

In [ ]:
audio_features.drop(columns=[
  'analysis_url',
], inplace=True)
audio_features.rename(columns={
  'id': 'audio_feature_id',
  'duration_ms': 'feature_duration_ms',
}, inplace=True)
audio_features.head()

### Mergear tablas

In [ ]:
merged_data = pd.merge(
  left = r_albums_tracks,
  right = tracks,
  how = 'inner',
  on = 'track_id',
)

merged_data = pd.merge(
  left = merged_data,
  right = albums,
  how = 'inner',
  on = 'album_id',
)

merged_data = pd.merge(
  left = r_albums_artists,
  right = merged_data,
  how = 'inner',
  on = 'album_id',
)

merged_data = pd.merge(
  left = merged_data,
  right = artists,
  how = 'inner',
  on = 'artist_id',
)

merged_data = pd.merge(
  left = merged_data,
  right = audio_features,
  how = 'inner',
  on = 'audio_feature_id',
)

In [ ]:
merged_data.head()

In [ ]:
merged_data.shape

In [ ]:
merged_data.info()

### Limitar/Filtrar data mergeada

Dentro de las modificaciones para el hito 2, consideramos limitar el periodo, la duracion y eliminar los albumes recopilatorios. \
Además, aqui tambien eliminaremos completamente los albumes que posean más de 30 canciones, eliminando uno de los casos que vimos durante la exploración.

In [ ]:
MIN_YEAR_TO_CONSIDER = 2000
MAX_YEAR_TO_CONSIDER = 2019
MIN_TRACK_DURATION = 1 * 60 * 1000 # 1 minuto como minimo
MAX_TRACK_DURATION = 6 * 60 * 1000 # 6 minutos como maximo
MAX_TRACK_NUMBER_IN_ALBUM = 30 # 30 canciones como maximo en un album (si no se cumple, se eliminan todas las canciones del album)
DELETED_TYPES = ['compilation']

merged_data = merged_data[
  (merged_data['album_release_year'] >= MIN_YEAR_TO_CONSIDER) &
  (merged_data['album_release_year'] <= MAX_YEAR_TO_CONSIDER) &
  (merged_data['track_duration_ms'] >= MIN_TRACK_DURATION) &
  (merged_data['track_duration_ms'] <= MAX_TRACK_DURATION) &
  (~merged_data['album_type'].isin(DELETED_TYPES))
]

removable_albums_ids = merged_data[
  merged_data['track_number_in_album'] > MAX_TRACK_NUMBER_IN_ALBUM
]['album_id'].unique()

merged_data = merged_data[
  ~merged_data['album_id'].isin(removable_albums_ids)
]

merged_data.drop_duplicates(subset=[
  'track_id', 'artist_id', 'album_id'
], inplace=True)


In [ ]:
merged_data.info()

### Guardado de datos en un csv

In [ ]:
# cambiar estos valores para guardar el dataframe
SAVE = False
if SAVE:
  merged_data.to_csv('data/merged_data.csv', index=False)

## Graficando

In [ ]:
# tomamos una muestra de los datos
fraction = 0.4
sampled_data = merged_data.sample(frac=fraction, random_state=42)
sampled_data.shape

In [ ]:
sampled_data['track_duration_s'] = sampled_data['track_duration_ms'] / 1000

In [ ]:
# histograma de la duracion de las canciones
sns.histplot(
  data = sampled_data,
  x = 'track_duration_s',
  bins = 100,
  kde = True,
)

In [ ]:
sns.histplot(
  data = sampled_data,
  x = 'track_popularity',
  bins = 25,
  kde = True,
)


In [ ]:
# plot release year
sns.histplot(
  data = sampled_data,
  x = 'album_release_year',
  bins = 20,
)

In [ ]:

# plot multiples histogramas (13 columnas/features)
fig, axs = plt.subplots(4, 4, figsize=(20, 20))

columns = sampled_data.columns[-13:]
for i, col in enumerate(columns):
  sns.histplot(
    data = sampled_data,
    x = col,
    ax = axs[i // 4][i % 4],
    bins = 25,
    kde = True,
  )


# merged_data.csv

`merged_data.csv` es el archivo resultante del procesamiento hecho en la seccion de `spotify.sqlite`, contiene todos los datos que consideramos luego de limpieza y seleccion de sus registros en un solo dataframe.

## carga

In [ ]:
merged_data = pd.read_csv('data/merged_data.csv')
merged_data.head()

In [ ]:
merged_data.shape

In [ ]:
merged_data.info()

# songs_normalize.csv

`songs_normalize.csv` es [un dataset de kaggle](https://www.kaggle.com/datasets/paradisejoy/top-hits-spotify-from-20002019) de los mejores hits de spotify, considerando desde 2000 al 2019.

## carga y modificaciones iniciales

In [3]:
songs_normalize = pd.read_csv('data/songs_normalize.csv')
songs_normalize.head()

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.7510,0.8340,1,-5.4440,0,0.0437,0.3000,0.0000,0.3550,0.8940,95.0530,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.4340,0.8970,0,-4.9180,1,0.0488,0.0103,0.0000,0.6120,0.6840,148.7260,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.5290,0.4960,7,-9.0070,1,0.0290,0.1730,0.0000,0.2510,0.2780,136.8590,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.5510,0.9130,0,-4.0630,0,0.0466,0.0263,0.0000,0.3470,0.5440,119.9920,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.6140,0.9280,8,-4.8060,0,0.0516,0.0408,0.0010,0.0845,0.8790,172.6560,pop


In [4]:
songs_normalize.shape

(2000, 18)

2000 filas y 18 columnas

In [5]:
songs_normalize.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist            2000 non-null   object 
 1   song              2000 non-null   object 
 2   duration_ms       2000 non-null   int64  
 3   explicit          2000 non-null   bool   
 4   year              2000 non-null   int64  
 5   popularity        2000 non-null   int64  
 6   danceability      2000 non-null   float64
 7   energy            2000 non-null   float64
 8   key               2000 non-null   int64  
 9   loudness          2000 non-null   float64
 10  mode              2000 non-null   int64  
 11  speechiness       2000 non-null   float64
 12  acousticness      2000 non-null   float64
 13  instrumentalness  2000 non-null   float64
 14  liveness          2000 non-null   float64
 15  valence           2000 non-null   float64
 16  tempo             2000 non-null   float64


Podemos tratar un poco la columna `genre`, para que sea una lista de generos para cada cancion

In [6]:
songs_normalize['genre'] = songs_normalize['genre'].apply(
  # split and then strip each genre
  lambda x: [i.strip() for i in x.split(',')]
)
songs_normalize['genre'].head()

0             [pop]
1       [rock, pop]
2    [pop, country]
3     [rock, metal]
4             [pop]
Name: genre, dtype: object

## analisis y tratamiento de nulos

In [7]:
print(f"=== Has NaN? ===")
for col in songs_normalize.columns:
  if col == 'genre':
    print(col, songs_normalize[col].apply(lambda x: len(x) == 0).any())
  else:
    print(col, songs_normalize[col].hasnans)

=== Has NaN? ===
artist False
song False
duration_ms False
explicit False
year False
popularity False
danceability False
energy False
key False
loudness False
mode False
speechiness False
acousticness False
instrumentalness False
liveness False
valence False
tempo False
genre False


## analisis de columnas

In [8]:
songs_normalize.describe()

,duration_ms,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000
mean,228748.1245,2009.4940,59.8725,0.6674,0.7204,5.3780,-5.5124,0.5535,0.1036,0.1290,0.0152,0.1812,0.5517,120.1226
std,39136.5690,5.8600,21.3356,0.1404,0.1527,3.6151,1.9335,0.4973,0.0962,0.1733,0.0878,0.1407,0.2209,26.9671
min,113000.0000,1998.0000,0.0000,0.1290,0.0549,0.0000,-20.5140,0.0000,0.0232,0.0000,0.0000,0.0215,0.0381,60.0190
25%,203580.0000,2004.0000,56.0000,0.5810,0.6220,2.0000,-6.4902,0.0000,0.0396,0.0140,0.0000,0.0881,0.3868,98.9857
50%,223279.5000,2010.0000,65.5000,0.6760,0.7360,6.0000,-5.2850,1.0000,0.0599,0.0557,0.0000,0.1240,0.5575,120.0215
75%,248133.0000,2015.0000,73.0000,0.7640,0.8390,8.0000,-4.1677,1.0000,0.1290,0.1762,0.0001,0.2410,0.7300,134.2655
max,484146.0000,2020.0000,89.0000,0.9750,0.9990,11.0000,-0.2760,1.0000,0.5760,0.9760,0.9850,0.8530,0.9730,210.8510


In [9]:
songs_normalize.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist            2000 non-null   object 
 1   song              2000 non-null   object 
 2   duration_ms       2000 non-null   int64  
 3   explicit          2000 non-null   bool   
 4   year              2000 non-null   int64  
 5   popularity        2000 non-null   int64  
 6   danceability      2000 non-null   float64
 7   energy            2000 non-null   float64
 8   key               2000 non-null   int64  
 9   loudness          2000 non-null   float64
 10  mode              2000 non-null   int64  
 11  speechiness       2000 non-null   float64
 12  acousticness      2000 non-null   float64
 13  instrumentalness  2000 non-null   float64
 14  liveness          2000 non-null   float64
 15  valence           2000 non-null   float64
 16  tempo             2000 non-null   float64


In [10]:
songs_normalize['explicit'].value_counts()

explicit
False    1449
True      551
Name: count, dtype: int64

In [11]:
songs_normalize['year'].value_counts().sort_index()

year
1998      1
1999     38
2000     74
2001    108
2002     90
2003     97
2004     96
2005    104
2006     95
2007     94
2008     97
2009     84
2010    107
2011     99
2012    115
2013     89
2014    104
2015     99
2016     99
2017    111
2018    107
2019     89
2020      3
Name: count, dtype: int64

Tenemos un buen balance de canciones que salieron en los 2000 - 2019, pero fuera de ese rango existe una cantidad significativa de canciones.

In [12]:
# obtain the genres
songs_normalize_genres = dict()
for genre_list in songs_normalize['genre']:
  for genre in genre_list:
    if genre not in songs_normalize_genres:
      songs_normalize_genres[genre] = 0
    songs_normalize_genres[genre] += 1

songs_normalize_genres = pd.DataFrame.from_dict(songs_normalize_genres, orient='index', columns=['count'])
songs_normalize_genres.sort_values(by='count', ascending=False, inplace=True)
songs_normalize_genres.head()

,count
pop,1633
hip hop,778
R&B,452
Dance/Electronic,390
rock,234


In [13]:
songs_normalize_genres.shape

(15, 1)

Solamente hay 15 generos

# charts.csv

`charts.csv` es [un dataset de kaggle](https://www.kaggle.com/datasets/dhruvildave/spotify-charts) que compila las colecciones "Top 200" y "50 virales" desde 2017 al 2021. Estas colecciones se publican globalmente cada 2-3 dias, dando informacion valiosa historica y alto valor de analisis.

## carga y modificaciones iniciales

In [14]:
charts = pd.read_csv('data/charts.csv')
charts.head()

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6lroV2Kg,Argentina,top200,SAME_POSITION,253019.0000
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul3ywMe46,Argentina,top200,MOVE_UP,223988.0000
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAOSC1qTfo,Argentina,top200,MOVE_DOWN,210943.0000
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtcMZ4S4bO,Argentina,top200,SAME_POSITION,173865.0000
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37DOZPJ2hf,Argentina,top200,MOVE_UP,153956.0000


In [15]:
charts.shape

(26173514, 9)

26.173.541 filas y 9 columnas

In [16]:
charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26173514 entries, 0 to 26173513
Data columns (total 9 columns):
 #   Column   Dtype  
---  ------   -----  
 0   title    object 
 1   rank     int64  
 2   date     object 
 3   artist   object 
 4   url      object 
 5   region   object 
 6   chart    object 
 7   trend    object 
 8   streams  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 1.8+ GB


Inicialmente notemos que hay columnas que podemos expresar como de tipo fechas y otras declarar como columnas categoricas.

In [17]:
charts['date'] = pd.to_datetime(charts['date'])
charts['region'] = charts['region'].astype('category')
charts['chart'] = charts['chart'].astype('category')
charts['trend'] = charts['trend'].astype('category')

In [18]:
charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26173514 entries, 0 to 26173513
Data columns (total 9 columns):
 #   Column   Dtype         
---  ------   -----         
 0   title    object        
 1   rank     int64         
 2   date     datetime64[ns]
 3   artist   object        
 4   url      object        
 5   region   category      
 6   chart    category      
 7   trend    category      
 8   streams  float64       
dtypes: category(3), datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 1.2+ GB


## analisis y tratamiento de nulos

In [19]:
print(f"=== Has NaN? ===")
for col in charts.columns:
  print(col, charts[col].hasnans)

=== Has NaN? ===
title True
rank False
date False
artist True
url False
region False
chart False
trend False
streams True


In [20]:
charts['title'].hasnans

True

`title` tiene valores faltantes, veamos cuantos son

In [21]:
charts[charts['title'].isna()].shape

(11, 9)

solamente 11 filas con titulo de cancion faltante veamos cuales son

In [22]:
charts[charts['title'].isna()]

,title,rank,date,artist,url,region,chart,trend,streams
7305002,NaN,120,2019-06-24,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,NEW_ENTRY,11942.0000
7401555,NaN,167,2019-06-25,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_DOWN,10310.0000
7479161,NaN,128,2019-06-26,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_UP,11620.0000
7563494,NaN,164,2019-06-27,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_DOWN,10229.0000
7670140,NaN,174,2019-06-28,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_DOWN,9348.0000
7772268,NaN,188,2019-06-29,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_DOWN,9808.0000
7879888,NaN,169,2019-06-30,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_UP,10900.0000
14391788,NaN,38,2019-07-15,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,viral50,NEW_ENTRY,NaN
14481594,NaN,48,2019-07-16,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,viral50,MOVE_DOWN,NaN
14768101,NaN,46,2019-07-17,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,viral50,MOVE_UP,NaN


Accediendo a la url y al artista por internet, podemos encontrar que existe una cancion llamada "NA", probablemente ocurrio algun error al ser cargadas. Podemos arreglarlo facilmente.

In [23]:
# fix the title "NA"
charts.loc[charts['title'].isna(), 'title'] = 'NA'

In [24]:
charts[charts['title'].isna()].shape

(0, 9)

In [25]:
charts[charts['title']=='NA']

,title,rank,date,artist,url,region,chart,trend,streams
7305002,NA,120,2019-06-24,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,NEW_ENTRY,11942.0000
7401555,NA,167,2019-06-25,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_DOWN,10310.0000
7479161,NA,128,2019-06-26,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_UP,11620.0000
7563494,NA,164,2019-06-27,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_DOWN,10229.0000
7670140,NA,174,2019-06-28,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_DOWN,9348.0000
7772268,NA,188,2019-06-29,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_DOWN,9808.0000
7879888,NA,169,2019-06-30,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,top200,MOVE_UP,10900.0000
14391788,NA,38,2019-07-15,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,viral50,NEW_ENTRY,NaN
14481594,NA,48,2019-07-16,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,viral50,MOVE_DOWN,NaN
14768101,NA,46,2019-07-17,Nissy,https://open.spotify.com/track/4cP6KmNvTFkLHZo6fVJq0c,Japan,viral50,MOVE_UP,NaN


In [26]:
charts['artist'].hasnans

True

`artist` tiene valores faltantes, veamos cuantos son

In [27]:
charts[charts['artist'].isna()].shape

(18, 9)

solamente 18 filas tienen artista faltante, veamos cuales son

In [28]:
charts[charts['artist'].isna()]

,title,rank,date,artist,url,region,chart,trend,streams
20596664,NO GOOD,10,2020-07-13,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,NEW_ENTRY,NaN
20616457,NO GOOD,10,2020-07-14,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20640094,NO GOOD,10,2020-07-15,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20661724,NO GOOD,10,2020-07-16,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20677645,NO GOOD,10,2020-07-17,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20705363,NO GOOD,10,2020-07-18,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20726697,NO GOOD,10,2020-07-19,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20748638,NO GOOD,13,2020-07-20,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,MOVE_DOWN,NaN
20788975,NO GOOD,14,2020-07-21,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,MOVE_DOWN,NaN
20833982,NO GOOD,19,2020-07-22,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,MOVE_DOWN,NaN


Buscando en internet la cancion, podemos identificar que el artista tiene nombre "N/A", probablemente ocurrio algun error al ser cargadas. Solucionemoslo.

In [29]:
# fix the artist name "N/A"
charts.loc[charts['artist'].isna(), 'artist'] = 'N/A'

In [30]:
charts[charts['artist'].isna()].shape

(0, 9)

In [31]:
charts[charts['artist']=='N/A']

,title,rank,date,artist,url,region,chart,trend,streams
20596664,NO GOOD,10,2020-07-13,N/A,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,NEW_ENTRY,NaN
20616457,NO GOOD,10,2020-07-14,N/A,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20640094,NO GOOD,10,2020-07-15,N/A,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20661724,NO GOOD,10,2020-07-16,N/A,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20677645,NO GOOD,10,2020-07-17,N/A,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20705363,NO GOOD,10,2020-07-18,N/A,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20726697,NO GOOD,10,2020-07-19,N/A,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,SAME_POSITION,NaN
20748638,NO GOOD,13,2020-07-20,N/A,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,MOVE_DOWN,NaN
20788975,NO GOOD,14,2020-07-21,N/A,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,MOVE_DOWN,NaN
20833982,NO GOOD,19,2020-07-22,N/A,https://open.spotify.com/track/4Qnz8tARYhUtDNePNsLx2C,Japan,viral50,MOVE_DOWN,NaN


In [32]:
charts['streams'].hasnans

True

`streams` tiene valores faltantes, veamos cuantos son

In [33]:
charts[charts['streams'].isna()].shape

(5851610, 9)

5.851.610 filas tienen `streams` faltantes, veamos a que colecciones pertenecen

In [34]:
charts[charts['streams'].isna()]['chart'].value_counts()

chart
viral50    5851610
top200           0
Name: count, dtype: int64

Todas las 5.851.610 filas son de la coleccion "50 Virales", comprobemos si es que esa coleccion no tiene `streams` en todos sus casos.

In [35]:
charts[charts['chart']=='viral50']['streams'].isna().value_counts()

streams
True    5851610
Name: count, dtype: int64

En efecto, las 5.851.610 filas que no poseen `streams` es debido a que las canciones de la coleccion "50 Virales" no poseen tal informacion.

## analisis de columnas

In [36]:
charts.describe()

,rank,date,streams
count,26173514.0000,26173514,20321904.0000
mean,80.9231,2019-08-20 15:35:30.458598912,55261.3144
min,1.0000,2017-01-01 00:00:00,1001.0000
25%,29.0000,2018-06-16 00:00:00,3546.0000
50%,67.0000,2019-09-18 00:00:00,9565.0000
75%,131.0000,2020-11-11 00:00:00,35409.0000
max,200.0000,2021-12-31 00:00:00,19749704.0000
std,59.1860,NaN,209587.2447


In [37]:
charts['region'].value_counts().sort_values(ascending=False)

region
Argentina        455308
United States    455057
Austria          454593
Brazil           454439
Australia        453103
                  ...  
Ukraine          127544
Russia           126837
Luxembourg        98053
Andorra           79592
South Korea       76276
Name: count, Length: 70, dtype: int64

In [38]:
charts['chart'].value_counts()

chart
top200     20321904
viral50     5851610
Name: count, dtype: int64

Los datos se encuentran en proporcion 200:50 = 4:1

In [39]:
charts['trend'].value_counts()

trend
MOVE_DOWN        11220434
MOVE_UP           9801048
SAME_POSITION     3298392
NEW_ENTRY         1853640
Name: count, dtype: int64

In [40]:
charts['title'].unique().shape, charts['artist'].unique().shape

((164807,), (96157,))

Tenemos 164.807 canciones y 96.157 artistas unicos

In [41]:
charts['date'].unique().shape

(1826,)

tenemos 1.826 fechas distintas

In [42]:
charts['date'].dt.year.value_counts().sort_index()

date
2017    4401145
2018    4863016
2019    5412669
2020    5793148
2021    5703536
Name: count, dtype: int64

En general, tenemos datos balanceados en los años

# data.csv

`data.csv` es un [dataset de kaggle](https://www.kaggle.com/datasets/ivannatarov/spotify-daily-top-200-songs-with-genres-20172021) que contiene la coleccion "Top 200" obtenida diariamente desde 2017 hasta 2021. \
El usuario que subio este dataset declara que es util para principiantes, dado que contiene cosas que se suelen tratar en la exploracion de datos.

## carga y modificaciones iniciales

In [43]:
data = pd.read_csv('data/data.csv', delimiter='#')
data.head()

,Position,Track Name,Artist,Streams,Date,Genre
0,1,Starboy,The Weeknd,3135625,2017-01-01,"['canadian pop', 'canadian contemporary r&b', 'pop']"
1,2,Closer,The Chainsmokers,3015525,2017-01-01,"['pop', 'pop dance', 'tropical house', 'edm', 'electropop', 'dance pop']"
2,3,Let Me Love You,DJ Snake,2545384,2017-01-01,"['pop', 'electronic trap', 'dance pop', 'edm', 'pop dance', 'pop rap']"
3,4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,2017-01-01,"['pop', 'uk dance', 'dance pop', 'uk funky', 'tropical house', 'pop dance', 'post-teen pop', 'edm']"
4,5,One Dance,Drake,2259887,2017-01-01,"['toronto rap', 'canadian pop', 'canadian hip hop', 'rap', 'pop rap', 'hip hop']"


In [44]:
data.shape

(321200, 6)

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321200 entries, 0 to 321199
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Position    321200 non-null  int64 
 1   Track Name  321182 non-null  object
 2   Artist      321182 non-null  object
 3   Streams     321200 non-null  int64 
 4   Date        321200 non-null  object
 5   Genre       321182 non-null  object
dtypes: int64(2), object(4)
memory usage: 14.7+ MB


In [46]:
data['Date'] = pd.to_datetime(data['Date'])

los generos estan como string para pandas, pero podemos notar que tiene nulos, tenemos que tratarlos primero

## analisis y tratamiento de nulos

In [47]:
print(f"=== Has NaN? ===")
for col in data.columns:
  print(col, data[col].hasnans)

=== Has NaN? ===
Position False
Track Name True
Artist True
Streams False
Date False
Genre True


In [48]:
data[data['Track Name'].isna()].shape

(18, 6)

In [49]:
data[data['Artist'].isna()].shape

(18, 6)

In [50]:
data[data['Genre'].isna()].shape

(18, 6)

es bastante sospechoso que sean 18 en todas estas columnas

In [51]:
data[data['Track Name'].isna() | data['Artist'].isna() | data['Genre'].isna()]

,Position,Track Name,Artist,Streams,Date,Genre
39204,5,NaN,NaN,3568811,2017-07-20,NaN
39212,13,NaN,NaN,2571960,2017-07-20,NaN
39229,30,NaN,NaN,1798208,2017-07-20,NaN
39238,39,NaN,NaN,1520291,2017-07-20,NaN
39404,5,NaN,NaN,3653533,2017-07-21,NaN
39415,16,NaN,NaN,2522453,2017-07-21,NaN
39434,35,NaN,NaN,1798890,2017-07-21,NaN
39447,48,NaN,NaN,1526955,2017-07-21,NaN
39700,101,NaN,NaN,747893,2017-07-22,NaN
39888,89,NaN,NaN,690247,2017-07-23,NaN


Lamentablemente no hay muchos datos que podamos sacar de esto sin las columnas que faltan

In [52]:
remove_data = data[data['Track Name'].isna() | data['Artist'].isna() | data['Genre'].isna()]
data.drop(remove_data.index, inplace=True)
del remove_data

print(f"=== Has NaN? ===")
for col in data.columns:
  print(col, data[col].hasnans)

=== Has NaN? ===
Position False
Track Name False
Artist False
Streams False
Date False
Genre False


Ahora que ya no hay nulos, trataremos la columna `Genre`

In [53]:
import re
def process_genres_str(x: str):
  # delete ' and " characters
  # delete the first and last character (they are [ and ])
  # split by , and then strip each genre
  return [i.strip() for i in re.sub(r'\'|\"', '', x)[1:-1].split(',')]

data['Genre'] = data['Genre'].apply(process_genres_str)
data['Genre'].head()

0                                         [canadian pop, canadian contemporary r&b, pop]
1                           [pop, pop dance, tropical house, edm, electropop, dance pop]
2                             [pop, electronic trap, dance pop, edm, pop dance, pop rap]
3    [pop, uk dance, dance pop, uk funky, tropical house, pop dance, post-teen pop, edm]
4                   [toronto rap, canadian pop, canadian hip hop, rap, pop rap, hip hop]
Name: Genre, dtype: object

## analisis de columnas

In [54]:
data.describe()

,Position,Streams,Date
count,321182.0000,321182.0000,321182
mean,100.5028,1188463.9883,2019-03-19 19:07:35.405346816
min,1.0000,325951.0000,2017-01-01 00:00:00
25%,51.0000,700483.2500,2018-02-10 00:00:00
50%,101.0000,895351.0000,2019-03-19 00:00:00
75%,151.0000,1364059.0000,2020-04-23 00:00:00
max,200.0000,17223237.0000,2021-07-17 00:00:00
std,57.7336,839659.5540,NaN


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 321182 entries, 0 to 321199
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Position    321182 non-null  int64         
 1   Track Name  321182 non-null  object        
 2   Artist      321182 non-null  object        
 3   Streams     321182 non-null  int64         
 4   Date        321182 non-null  datetime64[ns]
 5   Genre       321182 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 17.2+ MB


In [56]:
data['Track Name'].unique().shape, data['Artist'].unique().shape

((5496,), (1127,))

5.496 canciones y 1.127 artistas unicos

In [57]:
data['Date'].dt.year.value_counts().sort_index()

Date
2017    72182
2018    73000
2019    73000
2020    72000
2021    31000
Name: count, dtype: int64

Tenemos un muy buen balance entre las canciones de 2017 - 2020, en 2021 faltan, pero es esperable dada la fecha maxima encontrada

In [58]:
# obtain the genres
data_genres = dict()
for genre_list in data['Genre']:
  for genre in genre_list:
    if genre not in data_genres:
      data_genres[genre] = 0
    data_genres[genre] += 1

data_genres = pd.DataFrame.from_dict(data_genres, orient='index', columns=['count'])
data_genres.sort_values(by='count', ascending=False, inplace=True)
data_genres.head()

,count
pop,146708
dance pop,89558
rap,66629
post-teen pop,64118
pop rap,48831


In [59]:
data_genres.shape

(636, 1)

tenemos 636 generos